#  Retrieval of COL data in tabular format

### Credits

This is a snippet for retrieving Cost of Living Data from Numbeo, available at https://www.numbeo.com/cost-of-living/

### Requirements

In [65]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [66]:
def get_col_data(city,currency):
    req = requests.get(f'https://www.numbeo.com/cost-of-living/in/{city}?displayCurrency={currency}')
    soup = BeautifulSoup(req.text, 'html.parser')
    mydivs = soup.findAll("table", {"class": "data_wide_table"})
    df = pd.read_html(str(mydivs), attrs={'class': 'data_wide_table'})[0]
    df['Section'] = None
    df['Section'] = df.apply(lambda x: x[0] if x[1] =='[ Edit ]' else None,axis=1)
    df['Section'] = df.Section.ffill(inplace=False)
    
    def split_range(r,pos):
        try:
            r = str.replace(str(r),u'\xa0', u'-')
            result = str(r).split('-')[pos]
        except IndexError:
            result = None
        return result
    
    df['RangeLow'] = df.apply(lambda x: split_range(x[2],0) if x[1] !='[ Edit ]' else None,axis=1)
    df['RangeHigh'] = df.apply(lambda x: split_range(x[2],1) if x[1] !='[ Edit ]' else None,axis=1)
    df['Value'] = df.apply(lambda x: split_range(x[1],0) if x[1] !='[ Edit ]' else None,axis=1)
    df['Unit'] = df.apply(lambda x: currency if x[1] !='[ Edit ]' else None,axis=1)
    
    
    df = df.drop([1,2],axis=1)
    df = df.set_index(0)
    df.loc[df.index.str.contains('%'),'Unit'] = '%'
    df.index.name = 'ProductService'
    df = df[df.Unit.notna()]
    return df

## Usage

Function get_col_data with arguments city (string, e.g. 'Tokyo' and currency (3 letter code, string, e.g. 'JPY')

In [67]:
get_col_data('Tokyo','JPY')

,Section,RangeLow,RangeHigh,Value,Unit
ProductService,,,,,
"Meal, Inexpensive Restaurant",Restaurants,800.00,1000.00,"1,000.00",JPY
"Meal for 2 People, Mid-range Restaurant, Three-course",Restaurants,3000.00,8000.00,"5,000.00",JPY
McMeal at McDonalds (or Equivalent Combo Meal),Restaurants,600.00,700.00,680.00,JPY
Domestic Beer (0.5 liter draught),Restaurants,300.00,600.00,400.00,JPY
Imported Beer (0.33 liter bottle),Restaurants,300.00,800.00,400.00,JPY
Cappuccino (regular),Restaurants,300.00,500.00,382.00,JPY
Coke/Pepsi (0.33 liter bottle),Restaurants,100.00,180.00,136.67,JPY
Water (0.33 liter bottle),Restaurants,99.00,120.00,104.00,JPY
"Milk (regular), (1 liter)",Markets,150.00,240.00,186.53,JPY
